## Export packet loss for a selected perfsonar server

In [1]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
from datetime import datetime
import math
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from matplotlib import gridspec
from datetime import datetime
import numpy as np
import pandas as pd

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

##  Select your link

In [2]:
site='UC'
packetLossServer = "192.170.227.160"

# site='IU'
# packetLossServer = "149.165.225.223"

# site='UIUC'
# packetLossServer = "72.36.96.4"

# site='ICCN'
# packetLossServer = "72.36.96.4"

## Get the data from Elasticsearch, and store the record based on its type

In [3]:
my_query = {
    "_source": ["dest","timestamp","packet_loss"],
    'query':{
            'bool':{
                   'must':[
                       {'term': {'_type': 'packet_loss_rate'}},
                       {'range': {'timestamp': {'gte': '20170101T000000Z', 'lt': '20170401T000000Z'}}},
                       {'term': {'src': packetLossServer}}
                   ]
            }
    }
}

scroll = scan(client=es, index="network_weather-2017*", query=my_query)


In [4]:
epoch = datetime.utcfromtimestamp(0)
count = 0
dest=[]
pl=[]
ts=[]
for res in scroll:
    count += 1
    dt=datetime.strptime(res['_source']['timestamp'], '%Y%m%dT%H%M%S+0000')
    ts.append( int((dt - epoch).total_seconds()) )
    dest.append(res['_source']['dest'])
    pl.append(res['_source']['packet_loss'])
#     if count<2: print(res, ts)
    if not count%100000: print(count)
print(count)

data = pd.DataFrame({'dest':dest,'packet_loss':pl,'timestamp':ts})

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1497006


### saving in cvs files

In [5]:
#data

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1497006 entries, 0 to 1497005
Data columns (total 3 columns):
dest           1497006 non-null object
packet_loss    1497006 non-null float64
timestamp      1497006 non-null int64
dtypes: float64(1), int64(1), object(1)
memory usage: 34.3+ MB


In [7]:
data.sort_values(by='timestamp',inplace=True)    # Sort the records based on time
data.to_csv("Data/PacketLoss" + '_' + packetLossServer +".csv")